In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets, Output, RadioButtons, Checkbox, Dropdown
from IPython.display import display
from scipy.special import erfc
from scipy.signal import upfirdn

# Function to calculate the number of errors in ASK modulation
def ask_errors_sin(k, M, nsamp, EbN0_db, matched_filter_type='Normal'):
    # Determine number of levels in ASK constellation
    L = 2**k
    
    # Calculate signal-to-noise ratio (SNR) in dB and linear scale
    SNR_db = EbN0_db - 10 * np.log10(nsamp / (2 * k))
    SNR_linear = 10 ** (SNR_db / 10)
    
    # Generate random ASK symbols and apply cosine pulse shaping
    x = 2 * np.floor(L * np.random.rand(M)) - L + 1
    h = np.cos(2 * np.pi * np.arange(1, nsamp + 1) / nsamp)
    h = h / np.sqrt(np.sum(h**2))
    
    # Apply pulse shaping filter using upsample-filter-downsample method
    y = upfirdn(h, x, up=nsamp)
    y = y[:M * nsamp]
    
    # Calculate noise variance and generate noise
    P_x = np.mean(y ** 2)
    noise_variance = P_x / SNR_linear
    noise = np.random.normal(0, np.sqrt(noise_variance), len(y))
    
    # Add noise to signal
    y_noisy = y + noise

    # Apply matched filter (normal or reversed) and sample at decision points
    matched = h[::-1] if matched_filter_type == 'Reversed' else h
    yrx = np.convolve(y_noisy, matched, mode='full')
    z = yrx[nsamp - 1:M * nsamp:nsamp]

    # Decision based on closest constellation levels
    levels = np.arange(-L + 1, L, 2)
    z_decided = levels[np.abs(levels[:, None] - z).argmin(axis=0)]

    # Return the number of symbol errors
    return np.count_nonzero(x != z_decided)

# Set experiment parameters
M = 10000
EbN0_db = np.arange(0, 21, 2)

# Create interactive UI components
checkbox_4qam = Checkbox(value=True, description='4-ASK')
checkbox_8qam = Checkbox(value=False, description='8-ASK')
checkbox_16qam = Checkbox(value=False, description='16-ASK')
nsamp_dropdown = Dropdown(options=[4, 8, 16, 32, 64], value=16, description='Samples per Symbol:')
matched_radio = RadioButtons(options=['Normal', 'Reversed'], value='Normal', description='Matched Filter:')
plot_output = Output()

# Plot selected modulations based on user input
def plot_selected_modulations(change):
    with plot_output:
        plot_output.clear_output(wait=True)
        nsamp = nsamp_dropdown.value
        matched_filter_type = matched_radio.value
        plt.figure(figsize=(10, 7))
        colors = {'4-ASK': 'red', '8-ASK': 'green', '16-ASK': 'blue'}

        for k, checkbox, color in zip([2, 3, 4], [checkbox_4qam, checkbox_8qam, checkbox_16qam], colors.values()):
            if checkbox.value:
                L = 2**k
                modulation_name = f'{L}-ASK'
                ber = np.zeros(len(EbN0_db))
                for index, eb_n0 in enumerate(EbN0_db):
                    ber[index] = ask_errors_sin(k, M, nsamp, eb_n0, matched_filter_type) / (M * np.log2(L))

                plt.semilogy(EbN0_db, ber, 'o', label=f'Experimental {modulation_name}', color=color)
                ber_theoretical = (((L-1)/L) * erfc(np.sqrt(10**(EbN0_db / 10) * (3 * np.log2(L)) / (L**2 - 1)))) / k
                plt.semilogy(EbN0_db, ber_theoretical, linestyle='-', label=f'Theoretical {modulation_name}', color=color)

        plt.grid(True, which='both')
        plt.xlabel("Eb/N0 (dB)")
        plt.ylabel("BER")
        plt.title(f'Theoretical and Experimental BER of ASK Modulations [nsamp={nsamp}]')
        plt.legend()
        plt.show()

# Attach the plot function to UI interactions
checkbox_4qam.observe(plot_selected_modulations, names='value')
checkbox_8qam.observe(plot_selected_modulations, names='value')
checkbox_16qam.observe(plot_selected_modulations, names='value')
nsamp_dropdown.observe(plot_selected_modulations, names='value')
matched_radio.observe(plot_selected_modulations, names='value')

# Set up the UI layout
inputs1 = widgets.HBox([nsamp_dropdown])
inputs2 = widgets.HBox([matched_radio], layout=Layout(margin="0 0 0 20px"))
inputs12 = widgets.HBox([inputs1, inputs2], layout=Layout(align_items='center'))
inputs3 = widgets.VBox([checkbox_4qam, checkbox_8qam, checkbox_16qam])

inputs = widgets.VBox([inputs12, inputs3])

ui = widgets.HBox([inputs], layout=Layout(align_items='center'))

# Display UI and initial plot
display(ui, plot_output)
plot_selected_modulations(None)


Output()